The following cell loads in all the libraries that we previously installed; this makes it so that we can access the functions in these libraries instead of having to code these capabilities from scratch.

In [17]:
import csv
import cv2
import numpy as np
import librosa
import librosa.display
import wave
import os
import matplotlib.pyplot as plt
import multiprocessing 
import pickle
from ipywidgets import interact, widgets
from IPython.display import Image, display, HTML
import glob
import time

The next cell defines the folder paths to both the source folder where the audio is stored and the folder where the generated images should go.

In [2]:
SOURCE_DATA_ROOT='../AudioData/'
GENERATED_DATA_ROOT='../GeneratedData'

def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))
style = {'description_width': 'initial'}

SourceFolderDropdown= widgets.Dropdown(options=listdir_nohidden(SOURCE_DATA_ROOT), description='Source for Training Data:',style=style)

display(SourceFolderDropdown)


Dropdown(description='Source for Training Data:', options=('../AudioData/Cats-Vs-Dogs', '../AudioData/Cats-Vs-…

The following cell defines the filename where the Spectrum Variables should be read from. This will load in a file that tells this script how to compute the spectrogram. If you want to make additional Spectrum Variables Files, use the  [SpectrumsSettingTool](SpectrumsSettingsTool.ipynb).

In [3]:
#Loading in the Spectrogram variables
SOURCE_FOLDER_TRAIN= SourceFolderDropdown.value
def listSpecVarFiles(path):
    return glob.glob(os.path.join(path, '*.SpecVar'))
  
FileName = widgets.Dropdown(options=listSpecVarFiles('.'), description='Spectrum Variables File:',style=style)
display(FileName)



Dropdown(description='Spectrum Variables File:', options=('./Standard.SpecVar',), style=DescriptionStyle(descr…

In [4]:
SpectrumVariables = pickle.load(open(FileName.value, "rb"))

This defines the function that creates logarithmic spectragrams of the audio file designated in the `src_path` into the folder in the `dst_path`.

In [5]:
def log_mel_spec_tfm(src_path, dst_path):
    #print(src_path)
    fname=src_path.split('/')[-1];
    x=[]
    sample_rate=0
    try:
        channels ,sample_rate_in = librosa.load(src_path,mono=False)
    except:
        print(fname+" Could not be computed!")
        return
    if(channels.ndim==1):
        channels= channels.reshape((1,(len(channels))))
    channelcounter=0
    for audio_data in channels:
        channelcounter+=1
        
        RESOLUTION=SpectrumVariables["RESOLUTION"] 
        
        N_FFT=SpectrumVariables["N_FFT"]
        HOP_LENGTH= SpectrumVariables["HOP_LENGTH"]
        FMIN=SpectrumVariables["FMIN"]
        FMAX=SpectrumVariables["FMAX"]
        N_MELS=SpectrumVariables["N_MELS"]
        POWER=SpectrumVariables["POWER"]
        #audio_data = librosa.resample(x, sample_rate_in, SAMPLE_RATE)
        mel_spec_power = librosa.feature.melspectrogram(audio_data, sr=sample_rate_in, n_fft=N_FFT, 
                                                        hop_length=HOP_LENGTH, 
                                                        n_mels=N_MELS, power=POWER,
                                                       fmin=FMIN,fmax=FMAX)
        mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max)
        image = mel_spec_db; # convert to float
        image -= image.min() # ensure the minimal value is 0.0
        image /= image.max() # maximum value in image is now 1.0
        image*=256;
        img = image.astype(np.uint8)
        colorPic = cv2.applyColorMap(img, cv2.COLORMAP_BONE)
        #cv2.imshow('dst_rt', colorPic)
        #cv2.waitKey(0)
        count=0
        for i in range(int(np.floor(colorPic.shape[1]/RESOLUTION))):
            count+=1
            startx=RESOLUTION*i
            stopx=RESOLUTION*(i+1)
            OutputImage = cv2.resize(colorPic[:,startx:stopx,:],(RESOLUTION,RESOLUTION))
            plt.imsave(os.path.join(dst_path,(fname.replace(".flac",'-').replace(".aif",'-').replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+'-ch-'+str(channelcounter)+ '.png')), OutputImage) 
        if(count==0):
            print(src_path)
            #print("WARNING: => Jumped A file. If you see this often, increas sampleRate or Hop length.")

To Preview what the previously defined 'log_mel_spec_tfm' function does, we've made a little preview window that show the spectrum for each of the sound files.

In [6]:
AudioFolderDropdown= widgets.Dropdown(options=listdir_nohidden(SOURCE_FOLDER_TRAIN), description='Audio Folder:',style=style)
display(AudioFolderDropdown)


Dropdown(description='Audio Folder:', options=('../AudioData/UrbanSound/siren', '../AudioData/UrbanSound/car_h…

In [7]:
AudioFileDropdown= widgets.Dropdown(options=listdir_nohidden(AudioFolderDropdown.value), description='Audio File:',style=style)
display(AudioFileDropdown)


Dropdown(description='Audio File:', options=('../AudioData/UrbanSound/siren/184623.wav', '../AudioData/UrbanSo…

The following folder is a wrapper function for the previous function, going through all the audio files in `IN_FOLDER`.

In [15]:
def ComputeSpectrograms(IN_FOLDER,OUT_FOLDER):
    print("I will print the file path to files that are too short to fit into one full picture.")
    SourceFoldersLabels = [f.path for f in os.scandir(IN_FOLDER) if f.is_dir()]
    for path in SourceFoldersLabels:
        FileList = [f.path for f in os.scandir(path) if f.is_file() and (f.name.endswith(".aif") or f.name.endswith(".flac") or f.name.endswith(".wav") or f.name.endswith(".m4a") or f.name.endswith(".mp3"))]
        Label = path.split('/')[-1]
        outFolder = os.path.join(OUT_FOLDER,Label)
        if not os.path.exists(outFolder):
            os.makedirs(outFolder)
        with multiprocessing.Pool(12) as p:
            p.starmap(log_mel_spec_tfm, [(f,outFolder) for f in FileList])
        print("Finished this class. Going to the next")
    print("Done generating spectra!")  

This moves the Spectrum Variables file into the output folder so that we can correspond the images with labels.

In [16]:
StartTime=time.time()
OUTPUT_FOLDER_TRAIN = os.path.join(GENERATED_DATA_ROOT,os.path.basename(SOURCE_FOLDER_TRAIN))
ComputeSpectrograms(SOURCE_FOLDER_TRAIN,OUTPUT_FOLDER_TRAIN)
pickle.dump(SpectrumVariables, open(os.path.join(OUTPUT_FOLDER_TRAIN,'Main.SpecVar'), "wb" ))
print("Notebook Done. It took us about",time.time()-StartTime,"seconds.")

I will print the file path to files that are too short to fit into one full picture.


/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa

../AudioData/UrbanSound/siren/98536.wav


/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


../AudioData/UrbanSound/siren/43806.wav
../AudioData/UrbanSound/siren/43806.wav


...

Finished this class. Going to the next


/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa

../AudioData/UrbanSound/jackhammer/88466.wav


/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa/filters.py:221: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/anaconda3/envs/ImageCalc/lib/python3.7/site-packages/librosa

Finished this class. Going to the next
Done generating spectra!
Notebook Done


In [14]:
SOURCE_FOLDER_TRAIN

'../AudioData/UrbanSound'